# Background

The approach in this library has the goal of making iterative calculations with thermodynamic models much more reliable. In some cases, more than 1000x faster without sacrificing any accuracy. The greatest speedups are possible for two-phase inputs where at least one of the variables is temperature or pressure

The core question to answer is: suppose I know two thermodynamic variables that are not temperature and density (say, pressure and enthalpy), how can I most efficiently determine the temperature and density, the independent variables of the equation of state?

Two enabling technologies are:

* **Superancillary equations**: A set of mathematical approximation functions, pre-calculated based on phase equilibrium calculations in extended precision arithmetic. These numerical functions are selected because they can represent the results of the phase equilibrium so accurately that the conventional phase equilibrium calculation can be replaced with a single functional evaluation, which is exceptionally fast (order of 10 ns instead of 10 us)
* **K-D trees**: An efficient numerical structure that allows for nearest neighbor lookup

For pure fluids, for a given set x, y of thermodynamic variables, the general approach for iterative calculations can be expressed as two simpler subproblems:

A. *Is the input single phase or does it correspond to equilibrium of two phases?*

B. *For a region of interest of the phase diagram, what is a good guess value for the temperature and density close to the final solution?*

Subproblem A is handled with superancillary equations. Internal iteration is required for some input pairs, but the iterations are based on the use of superancillary functions, so the subiterations are reliable and fast. This represents a generational improvement in the reliability and speed of phase determination from thermodynamic models

After subproblem A is complete, the phase of the inputs is known. If the inputs are two-phase, the overall problem is finished.

Next comes subproblem B. When the thermodynamic model was loaded, "lighthouse points" were densely populated throughout the entire single-phase portion of the phase diagram. These points were then passed into a K-D tree nearest-neighbor lookup structure. The entire single-phase region is partitioned into non-overlapping and non-intersecting regions.

In subproblem B, the regions are queried for their nearest point closest to the values satisfying the constraint equations. The lookup is very fast (order of ns), and once this guess value is known, a traditional Newton iteration is carried out to obtain the solution. If the routines are not reliable enough, the algorithm can be made arbitrarily reliable by increasing the number of points in the K-D tree.  Of course, that reliability comes at the cost of additional memory required for the storage.